In [ ]:
columns = ["X","Y","A"]
Germany_id='DFL-CLU-000N8Y'
Germany = positions.loc[
    # index (rows) - start:end
    :,
    # columns
    (
        Germany_id, # team level
        slice(None),   # player level
        columns    # position level
    )
]

In [ ]:
dfa=Germany[Germany_id,'DFL-OBJ-000191','A'][Germany[Germany_id,'DFL-OBJ-000191','A']>5]  ##filter the team df for acceleration higher than 5

In [ ]:
startframe=[]   ##initialize the frames where the player sprints
endframe=[]
for idx, val in enumerate(dfa):
    if idx==0:
         startframe.append(dfa.index.get_level_values('N')[idx])
    if 0<idx and idx<dfa.count()-2:
        if float(dfa.index.get_level_values('N')[idx+1])-float(dfa.index.get_level_values('N')[idx]) > 100:
            endframe.append(dfa.index.get_level_values('N')[idx+1])
            startframe.append(dfa.index.get_level_values('N')[idx+2])
    if idx==dfa.count()-2:
        if float(dfa.index.get_level_values('N')[idx+1])-float(dfa.index.get_level_values('N')[idx]) > 100:
            endframe.append(dfa.index.get_level_values('N')[idx+1])
    if idx==dfa.count()-1:
        del startframe[-1]

In [ ]:
sprintframes = {'Startframe': startframe,
                 'Endframe': endframe}
sprintframesdf = pd.DataFrame(sprintframes,columns= ['Startframe', 'Endframe'])
print(sprintframesdf)